# Bu ödevde, önceki derste Titanic verileri ile oluşturduğunuz modelin performansını ölçeceksiniz. Bu ödevi tamamlamak için, aşağıdaki görevlerin çözümlerini içeren bir Jupyter not defterini link olarak gönderin. Kafanıza takılan konuları mentörünüzle konuşabilir ya da ofis saatinde Slack üzerinden sorabilirsiniz.

Modelinizin performansı çapraz doğrulama yaparak ve farklı metrikler kullanarak değerlendirin.
Hiperparemetre ayarlaması yaparak en uygun parametrelere sahip modeli belirleyin.

In [75]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import warnings 
warnings.filterwarnings("ignore")

In [76]:
titanic= pd.read_csv("Desktop/Bootcamp Ödevler/sınıflandırma/train.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [77]:
titanic["Age"].fillna(titanic["Age"].mean(), inplace=True)

In [78]:
kategoriler= ["Embarked", "Sex"]
df= pd.concat([titanic, pd.get_dummies(titanic[kategoriler])], axis=1) 
df.drop(kategoriler, inplace=True, axis=1)
df.head()

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,0,0,1,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,1,0,0,1,0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,0,0,1,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,0,0,1,1,0
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,0,0,1,0,1


### Logistic Regression Model

In [79]:
ozellikler= ["Pclass","Age", "Embarked_C", "Embarked_Q", "Embarked_S", "Sex_female", "Sex_male"]
y= df["Survived"]
X= df[ozellikler]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=111)

log_reg= LogisticRegression()
log_reg.fit(X_train, y_train)
print("Eğitim kümesi doğruluk değeri: {}".format(log_reg.score(X_train, y_train)))
print("Test kümesi doğruluk değeri: {}".format(log_reg.score(X_test, y_test )))

Eğitim kümesi doğruluk değeri: 0.800561797752809
Test kümesi doğruluk değeri: 0.7597765363128491


In [80]:
tahmin_egitim= log_reg.predict(X_train)
tahmin_test= log_reg.predict(X_test)

In [81]:
from sklearn.metrics import classification_report

In [82]:
print(classification_report(y_test, tahmin_test))

              precision    recall  f1-score   support

           0       0.81      0.81      0.81       113
           1       0.67      0.68      0.68        66

    accuracy                           0.76       179
   macro avg       0.74      0.74      0.74       179
weighted avg       0.76      0.76      0.76       179



#### Çapraz doğrulama

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=111, stratify=y)

print("Tüm veri kümesi 0 yüzdesi: ", (len(y[y==0])/len(y)))
print("Eğitim kümesi 0 yüzdesi: ", (len(y_train[y_train==0])/len(y_train)))
print("Test kümesi 0 yüzdesi: ", (len(y_test[y_test==0])/len(y_test)))

Tüm veri kümesi 0 yüzdesi:  0.6161616161616161
Eğitim kümesi 0 yüzdesi:  0.6165730337078652
Test kümesi 0 yüzdesi:  0.6145251396648045


In [84]:
log_reg_1= LogisticRegression()
log_reg_1.fit(X_train, y_train)
tahmin_egitim= log_reg_1.predict(X_train)
tahmin_test= log_reg_1.predict(X_test)
log_reg_1.score(X_test, Y_test)

0.547486033519553

Cross Validate ile çapraz doğrulama

In [85]:
from sklearn.model_selection import cross_validate, cross_val_score

In [96]:
lrm= LogisticRegression()
cv= cross_validate(estimator=lrm,
                   X=X,
                   y=y,
                   cv=10)
print("Test skorları: ", cv["test_score"], sep="\n")
print("Test kümesi ortalaması", cv["test_score"].mean())
# print("Eğitim skorları: ", cv["train_score"], sep="\n") çalışmıyor!!!


Test skorları: 
[0.78888889 0.78651685 0.76404494 0.85393258 0.7752809  0.78651685
 0.7752809  0.78651685 0.79775281 0.79775281]
Test kümesi ortalaması 0.7912484394506867


In [87]:
cv= cross_validate(estimator=lrm,
                   X=X,
                   y=y,
                   cv=10,
                   scoring= ["accuracy", "precision", "r2"])
print("Tesst kümesi doğruluk ortalaması: ", cv["test_accuracy"].mean())
print("Test kümesi hassasiyet ortalaması: ", cv["test_precision"].mean())
print("Test kümesi r kare değeri: ", cv["test_r2"].mean())

Tesst kümesi doğruluk ortalaması:  0.7912484394506867
Test kümesi hassasiyet ortalaması:  0.7395090490374466
Test kümesi r kare değeri:  0.1172625413801884


model tahminleri

In [88]:
from sklearn.model_selection import cross_val_predict

In [89]:
y_tahmin= cross_val_predict(estimator=lrm, X=X, y=y, cv=10)
print(y_tahmin[0:10])

[0 1 1 1 0 0 0 0 1 1]


# Hiperparametre Ayarlama:

In [90]:
from sklearn.model_selection import GridSearchCV

In [91]:
parametreler= {"C": [10**x for x in range(-5,5,1)],
               "penalty": ["l1", "l2"]}
gs= GridSearchCV(estimator=log_reg,
                 param_grid= parametreler,
                 cv=10
                 )
gs.fit(X,y)

GridSearchCV(cv=10, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000, 10000],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [92]:
print("En iyi parametreler: ", gs.best_params_)
print("En iyi skor: ", gs.best_score_)

En iyi parametreler:  {'C': 0.1, 'penalty': 'l2'}
En iyi skor:  0.7957553058676654


RandomizedSearchCV:

In [93]:
from sklearn.model_selection import RandomizedSearchCV

In [94]:
parametreler= {"C": [10**x for x in range(-5,5,1)],
               "penalty": ["l1", "l2"]}
rs= RandomizedSearchCV(estimator=log_reg,
                       param_distributions= parametreler,
                       cv=10,
                       n_iter=10, 
                       random_state=111,
                       scoring= "accuracy")
rs.fit(X,y)

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=LogisticRegression(C=1.0, class_weight=None,
                                                dual=False, fit_intercept=True,
                                                intercept_scaling=1,
                                                l1_ratio=None, max_iter=100,
                                                multi_class='auto', n_jobs=None,
                                                penalty='l2', random_state=None,
                                                solver='lbfgs', tol=0.0001,
                                                verbose=0, warm_start=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1,
                                              1, 10, 100, 1000, 10000],
                                        'penalty': ['l1', 'l2']},
                   pre_dispatch='2*n_jobs', random_state=111, r

In [95]:
print("En iyi parametreler: ", rs.best_params_)
print("En iyi skor: ", rs.best_score_)
print("Tüm accuracy değerleri: ", rs.cv_results_["mean_test_score"])

En iyi parametreler:  {'penalty': 'l2', 'C': 1}
En iyi skor:  0.7912484394506867
Tüm accuracy değerleri:  [0.61616729 0.61838951 0.78901373 0.79124844        nan 0.79013733
        nan        nan        nan        nan]
